# NGram Model to play Hangman

Design an algorithm for Hangman using a neural network that learns from a training set of 250,000 words. It predicts the next best letter to guess based on hidden word patterns and previous guesses, achieving more than 50% accuracy in testing.


Note - This is done via the TrexQuant API

In [ ]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from collections import defaultdict, Counter
import pandas as pd
import numpy as np

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

## Preprocessing the data

I am reading in all the words in the given dataset for preprocessing it into the n grams model.
(This could have been done inside the class too, but it takes some time to execute, so I thought of writing it on top to save time every time I start a new game.)

In [2]:
# Reading all words
f = open("words_250000_train.txt")

all_words = []
length_dict = Counter()

lines = f.readlines()
for line in lines:
    word = line.strip()
    length_dict[len(word)] +=1
    all_words.append(word)
f.close()


# These are the number of words for different lenghts
print('Total words =', len(all_words))
for i in sorted(length_dict):
    print(i, length_dict[i])

Total words = 227300
1 17
2 264
3 2201
4 5287
5 11274
6 19541
7 25948
8 30452
9 30906
10 26953
11 22786
12 18178
13 12956
14 8710
15 5211
16 3143
17 1775
18 859
19 441
20 225
21 98
22 44
23 14
24 9
25 3
27 2
28 1
29 2


In [3]:
# Statistical inferences from the distribution of lengths of words
df = pd.DataFrame(list(length_dict.items()), columns=['num', 'freq'])
expanded_data = np.repeat(df['num'], df['freq'])

mean = np.mean(expanded_data)
print('Mean', mean)
median = np.median(expanded_data)
print('Median', median)
std_dev = np.std(expanded_data)
print('Mean', mean)



Mean 9.347760668719754
Median 9.0
Mean 9.347760668719754


I have implemented the N-gram model with N = 7 for this problem

In [4]:
def sevengram(corpus):

        ## Initializing the n-gram counter dictionaries
        unigram_counts = defaultdict(Counter)
        #print(unigram_counts)
        bigram_counts_second = defaultdict(Counter)
        bigram_counts_first = defaultdict(Counter)

        trigram_counts_third = defaultdict(Counter)
        trigram_counts_second = defaultdict(Counter)
        trigram_counts_first = defaultdict(Counter)

        fourgram_counts_first = defaultdict(Counter)
        fourgram_counts_second = defaultdict(Counter)
        fourgram_counts_third = defaultdict(Counter)
        fourgram_counts_fourth = defaultdict(Counter)

        fivegram_counts_first = defaultdict(Counter)
        fivegram_counts_second = defaultdict(Counter)
        fivegram_counts_third = defaultdict(Counter)
        fivegram_counts_fourth = defaultdict(Counter)
        fivegram_counts_fifth = defaultdict(Counter)

        sixgram_counts_first = defaultdict(Counter)
        sixgram_counts_second = defaultdict(Counter)
        sixgram_counts_third = defaultdict(Counter)
        sixgram_counts_fourth = defaultdict(Counter)
        sixgram_counts_fifth = defaultdict(Counter)
        sixgram_counts_sixth = defaultdict(Counter)

        sevengram_counts_first = defaultdict(Counter)
        sevengram_counts_second = defaultdict(Counter)
        sevengram_counts_third = defaultdict(Counter)
        sevengram_counts_fourth = defaultdict(Counter)
        sevengram_counts_fifth = defaultdict(Counter)
        sevengram_counts_sixth = defaultdict(Counter)
        sevengram_counts_seventh = defaultdict(Counter)


        # Generate a list of unigram_counts
        for word in corpus:
            length = len(word)
            for char in word:
                #index will be[word's length][character]
                unigram_counts[length][char] += 1

        for key in unigram_counts.keys():
            if not len(unigram_counts[key]) == 26:
                add_char = set(string.ascii_lowercase) - set(list(unigram_counts[key].keys()))

                for char in add_char:
                    unigram_counts[key][char] = 0
        


        for word in corpus:
            word = "$$$$$$" + word + "######"

            # generate a list of bigrams
            bigram_list = zip(word, word[1:])

            # generate a list of trigrams
            trigram_list = zip(word, word[1:], word[2:])

            # generate a list of fourgrams
            fourgram_list = zip(word, word[1:], word[2:], word[3:])

            # generate a list of fivegrams
            fivegram_list = zip(word, word[1:], word[2:], word[3:], word[4:])

            # generate a list of sixgrams
            sixgram_list = zip(word, word[1:], word[2:], word[3:], word[4:], word[5:])

            # generate a list of sevengrams
            sevengram_list = zip(word, word[1:], word[2:], word[3:], word[4:], word[5:], word[6:])

            # iterate over bigrams
            for bigram in bigram_list:
                first, second = bigram
                bigram_counts_second[first][second] += 1
                bigram_counts_first[second][first] += 1
            bigram_counts = [bigram_counts_first, bigram_counts_second]



            # iterate over trigrams
            for trigram in trigram_list:
                first, second, third = trigram
                trigram_counts_third[first+second][third] += 1
                trigram_counts_second[first+third][second] += 1
                trigram_counts_first[second+third][first] += 1
            trigram_counts = [trigram_counts_first, trigram_counts_second, trigram_counts_third]


            # iterate over fourgrams
            for fourgram in fourgram_list:
                first, second, third, fourth = fourgram
                fourgram_counts_fourth[first+second+third][fourth] += 1
                fourgram_counts_third[first+second+fourth][third] += 1
                fourgram_counts_second[first+third+fourth][second] += 1
                fourgram_counts_first[second+third+fourth][first] += 1
            fourgram_counts = [fourgram_counts_first, fourgram_counts_second, fourgram_counts_third, fourgram_counts_fourth]


            # iterate over fivegrams
            for fivegram in fivegram_list:
                first, second, third, fourth, fifth = fivegram
                fivegram_counts_fifth[first+second+third+fourth][fifth] += 1
                fivegram_counts_fourth[first+second+third+fifth][fourth] += 1
                fivegram_counts_third[first+second+fourth+fifth][third] += 1
                fivegram_counts_second[first+third+fourth+fifth][second] += 1
                fivegram_counts_first[second+third+fourth+fifth][first] += 1
            fivegram_counts = [fivegram_counts_first, fivegram_counts_second, fivegram_counts_third, fivegram_counts_fourth, fivegram_counts_fifth]


            # iterate over sixgrams
            for sixgram in sixgram_list:
                first, second, third, fourth, fifth, sixth = sixgram
                sixgram_counts_sixth[first+second+third+fourth+fifth][sixth] += 1
                sixgram_counts_fifth[first+second+third+fourth+sixth][fifth] += 1
                sixgram_counts_fourth[first+second+third+fifth+sixth][fourth] += 1
                sixgram_counts_third[first+second+fourth+fifth+sixth][third] += 1
                sixgram_counts_second[first+third+fourth+fifth+sixth][second] += 1
                sixgram_counts_first[second+third+fourth+fifth+sixth][first] += 1
            sixgram_counts = [sixgram_counts_first, sixgram_counts_second, sixgram_counts_third, sixgram_counts_fourth, sixgram_counts_fifth, sixgram_counts_sixth]

            
            # iterate over sevengrams
            for sevengram in sevengram_list:
                first, second, third, fourth, fifth, sixth, seventh = sevengram
                sevengram_counts_seventh[first+second+third+fourth+fifth+sixth][seventh] += 1
                sevengram_counts_sixth[first+second+third+fourth+fifth+seventh][sixth] += 1
                sevengram_counts_fifth[first+second+third+fourth+sixth+seventh][fifth] += 1
                sevengram_counts_fourth[first+second+third+fifth+sixth+seventh][fourth] += 1
                sevengram_counts_third[first+second+fourth+fifth+sixth+seventh][third] += 1
                sevengram_counts_second[first+third+fourth+fifth+sixth+seventh][second] += 1
                sevengram_counts_first[second+third+fourth+fifth+sixth+seventh][first] += 1
            sevengram_counts = [sevengram_counts_first, sevengram_counts_second, sevengram_counts_third, sevengram_counts_fourth, sevengram_counts_fifth, sevengram_counts_sixth, sevengram_counts_seventh]


        return unigram_counts, bigram_counts, trigram_counts, fourgram_counts, fivegram_counts, sixgram_counts, sevengram_counts
    

In [5]:
unigram_counts, bigram_counts, trigram_counts, fourgram_counts, fivegram_counts, sixgram_counts, sevengram_counts = sevengram(all_words)

## Actual Program

In [6]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []

        # These are the counts calculated in 'sevengram' function itself.
        self.unigram_counts = unigram_counts
        self.bigram_counts = bigram_counts
        self.trigram_counts = trigram_counts
        self.fourgram_counts = fourgram_counts
        self.fivegram_counts = fivegram_counts
        self.sixgram_counts = sixgram_counts
        self.sevengram_counts = sevengram_counts

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word):

        def ngram_prob(key, char, ngram_counts):
            if float(sum(ngram_counts[key].values()))==0:
                return 0
            return ngram_counts[key][char] / float(sum(ngram_counts[key].values()))

        clean_word = word[::2]
        len_word = len(clean_word)
        n = len_word
        current_dictionary = self.current_dictionary

        available = list(set(string.ascii_lowercase) - set(self.guessed_letters))
        sevengram_probs = []

        unigram_counts = self.unigram_counts
        bigram_counts = self.bigram_counts
        trigram_counts = self.trigram_counts
        fourgram_counts = self.fourgram_counts
        fivegram_counts = self.fivegram_counts
        sixgram_counts = self.sixgram_counts
        sevengram_counts = self.sevengram_counts

        unigram_counts = unigram_counts
        bigram_counts_first, bigram_counts_second = bigram_counts
        trigram_counts_first, trigram_counts_second, trigram_counts_third = trigram_counts
        fourgram_counts_first, fourgram_counts_second, fourgram_counts_third, fourgram_counts_fourth = fourgram_counts
        fivegram_counts_first, fivegram_counts_second, fivegram_counts_third, fivegram_counts_fourth, fivegram_counts_fifth = fivegram_counts
        sixgram_counts_first, sixgram_counts_second, sixgram_counts_third, sixgram_counts_fourth, sixgram_counts_fifth, sixgram_counts_sixth = sixgram_counts
        sevengram_counts_first, sevengram_counts_second, sevengram_counts_third, sevengram_counts_fourth, sevengram_counts_fifth, sevengram_counts_sixth, sevengram_counts_seventh = sevengram_counts

        

        mask = ['$', '$', '$', '$', '$', '$'] + list(clean_word) + ['#', '#', '#', '#', '#', '#']
        for char in available:
            char_prob = 0
            for index in range(6,n+6):
                prob1, prob2, prob3, prob4, prob5, prob6, prob7 = 0, 0, 0, 0, 0, 0, 0

                # The first case is that the char has not been guessed
                if mask[index] == '_':

                    # Case 1
                    if not mask[index+1] == '_':
                        if not mask[index+2] == '_':
                            if not mask[index+3] == '_':
                                if not mask[index+4] == '_':
                                    if not mask[index+5] == '_':
                                        if not mask[index+6] == '_':
                                            prob1 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3]+mask[index+4]+mask[index+5]+mask[index+6], char, sevengram_counts_first)
                                        else:
                                            prob1 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3]+mask[index+4]+mask[index+5], char, sixgram_counts_first)
                                    else:
                                        prob1 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3]+mask[index+4], char, fivegram_counts_first)
                                else:
                                    prob1 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3], char, fourgram_counts_first)
                            else:
                                prob1 = ngram_prob(mask[index+1]+mask[index+2], char, trigram_counts_first)
                        else:
                            prob1 = ngram_prob(mask[index+1], char, bigram_counts_first)
                    else:
                        prob1 = ngram_prob(n, char, unigram_counts)

                    
                    # Case 2
                    if not mask[index-1] == '_':
                        if not mask[index+1] == '_':
                            if not mask[index+2] == '_':
                                if not mask[index+3] == '_':
                                    if not mask[index+4] == '_':
                                        if not mask[index+5] == '_':
                                            prob2 = ngram_prob(mask[index-1]+mask[index+1]+mask[index+2]+mask[index+3]+mask[index+4]+mask[index+5], char, sevengram_counts_second)
                                        else:
                                            prob2 = ngram_prob(mask[index-1]+mask[index+1]+mask[index+2]+mask[index+3]+mask[index+4], char, sixgram_counts_second)
                                    else:
                                        prob2 = ngram_prob(mask[index-1]+mask[index+1]+mask[index+2]+mask[index+3], char, fivegram_counts_second)
                                else:
                                    prob2 = ngram_prob(mask[index-1]+mask[index+1]+mask[index+2], char, fourgram_counts_second)
                            else:
                                prob2 = ngram_prob(mask[index-1]+mask[index+1], char, trigram_counts_second)
                        else:
                            prob2 = ngram_prob(mask[index-1], char, bigram_counts_second)
                    else:
                        if not mask[index+1] == '_':
                            if not mask[index+2] == '_':
                                if not mask[index+3] == '_':
                                    if not mask[index+4] == '_':
                                        if not mask[index+5] == '_':
                                            prob2 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3]+mask[index+4]+mask[index+5], char, sixgram_counts_first)
                                        else:
                                            prob2 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3]+mask[index+4], char, fivegram_counts_first)
                                    else:
                                        prob2 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3], char, fourgram_counts_first)
                                else:
                                    prob2 = ngram_prob(mask[index+1]+mask[index+2], char, trigram_counts_first)
                            else:
                                prob2 = ngram_prob(mask[index+1], char, bigram_counts_first)
                        else:
                            prob2 = ngram_prob(n, char, unigram_counts)

                    # Case 3
                    if not mask[index-2] == '_':
                        if not mask[index-1] == '_':
                            if not mask[index+1] == '_':
                                if not mask[index+2] == '_':
                                    if not mask[index+3] == '_':
                                        if not mask[index+4] == '_':
                                            prob3 = ngram_prob(mask[index-2]+mask[index-1]+mask[index+1]+mask[index+2]+mask[index+3]+mask[index+4], char, sevengram_counts_third)
                                        else:
                                            prob3 = ngram_prob(mask[index-2]+mask[index-1]+mask[index+1]+mask[index+2]+mask[index+3], char, sixgram_counts_third)
                                    else:
                                        prob3 = ngram_prob(mask[index-2]+mask[index-1]+mask[index+1]+mask[index+2], char, fivegram_counts_third)
                                else:
                                    prob3 = ngram_prob(mask[index-2]+mask[index-1]+mask[index+1], char, fourgram_counts_third)
                            else:
                                prob3 = ngram_prob(mask[index-2]+mask[index-1], char, trigram_counts_third)
                        else:
                            if not mask[index+1] == '_':
                                if not mask[index+2] == '_':
                                    if not mask[index+3] == '_':
                                        if not mask[index+4] == '_':
                                            prob3 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3]+mask[index+4], char, fivegram_counts_first)
                                        else:
                                            prob3 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3], char, fourgram_counts_first)
                                    else:
                                        prob3 = ngram_prob(mask[index+1]+mask[index+2], char, trigram_counts_first)
                                else:
                                    prob3 = ngram_prob(mask[index+1], char, bigram_counts_first)
                            else:
                                prob3 = ngram_prob(n, char, unigram_counts)
                    else:
                        if not mask[index-1] == '_':
                            if not mask[index+1] == '_':
                                if not mask[index+2] == '_':
                                    if not mask[index+3] == '_':
                                        if not mask[index+4] == '_':
                                            prob3 = ngram_prob(mask[index-1]+mask[index+1]+mask[index+2]+mask[index+3]+mask[index+4], char, sixgram_counts_second)
                                        else:
                                            prob3 = ngram_prob(mask[index-1]+mask[index+1]+mask[index+2]+mask[index+3], char, fivegram_counts_second)
                                    else:
                                        prob3 = ngram_prob(mask[index-1]+mask[index+1]+mask[index+2], char, fourgram_counts_second)
                                else:
                                    prob3 = ngram_prob(mask[index-1]+mask[index+1], char, trigram_counts_second)
                            else:
                                prob3 = ngram_prob(mask[index-1], char, bigram_counts_second)
                        else:
                            if not mask[index+1] == '_':
                                if not mask[index+2] == '_':
                                    if not mask[index+3] == '_':
                                        if not mask[index+4] == '_':
                                            prob3 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3]+mask[index+4], char, fivegram_counts_first)
                                        else:
                                            prob3 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3], char, fourgram_counts_first)
                                    else:
                                        prob3 = ngram_prob(mask[index+1]+mask[index+2], char, trigram_counts_first)
                                else:
                                    prob3 = ngram_prob(mask[index+1], char, bigram_counts_first)
                            else:
                                prob3 = ngram_prob(n, char, unigram_counts)

                    # Case 4
                    if not mask[index-1] == '_':
                        if not mask[index-2] == '_':
                                if not mask[index-3] == '_':
                                    if not mask[index+1] == '_':
                                        if not mask[index+2] == '_':
                                            if not mask[index+3] == '_':
                                                prob4 = ngram_prob(mask[index-3]+mask[index-2]+mask[index-1]+mask[index+1]+mask[index+2]+mask[index+3], char, sevengram_counts_fourth)
                                            else:
                                                prob4 = ngram_prob(mask[index-3]+mask[index-2]+mask[index-1]+mask[index+1]+mask[index+2], char, sixgram_counts_fourth)
                                        else:
                                            prob4 = ngram_prob(mask[index-3]+mask[index-2]+mask[index-1]+mask[index+1], char, fivegram_counts_fourth)
                                    else:
                                        prob4 = ngram_prob(mask[index-3]+mask[index-2]+mask[index-1], char, fourgram_counts_fourth)
                                else:
                                    if not mask[index+1] == '_':
                                        if not mask[index+2] == '_':
                                            if not mask[index+3] == '_':
                                                prob4 = ngram_prob(mask[index-2]+mask[index-1]+mask[index+1]+mask[index+2]+mask[index+3], char, sixgram_counts_third)
                                            else:
                                                prob4 = ngram_prob(mask[index-2]+mask[index-1]+mask[index+1]+mask[index+2], char, fivegram_counts_third)
                                        else:
                                            prob4 = ngram_prob(mask[index-2]+mask[index-1]+mask[index+1], char, fourgram_counts_third)
                                    else:
                                        prob4 = ngram_prob(mask[index-2]+mask[index-1], char, trigram_counts_third)
                        else:
                            if not mask[index+1] == '_':
                                if not mask[index+2] == '_':
                                    if not mask[index+3] == '_':
                                        prob4 = ngram_prob(mask[index-1]+mask[index+1]+mask[index+2]+mask[index+3], char, fivegram_counts_second)
                                    else:
                                        prob4 = ngram_prob(mask[index-1]+mask[index+1]+mask[index+2], char, fourgram_counts_second)
                                else:
                                    prob4 = ngram_prob(mask[index-1]+mask[index+1], char, trigram_counts_second)
                            else:
                                prob4 = ngram_prob(mask[index-1], char, bigram_counts_second)
                    else:
                        if not mask[index+1] == '_':
                            if not mask[index+2] == '_':
                                if not mask[index+3] == '_':
                                    prob4 = ngram_prob(mask[index+1]+mask[index+2]+mask[index+3], char, fourgram_counts_first)
                                else:
                                    prob4 = ngram_prob(mask[index+1]+mask[index+2], char, trigram_counts_first)
                            else:
                                prob4 = ngram_prob(mask[index+1], char, bigram_counts_first)
                        else:
                            prob4 = ngram_prob(n, char, unigram_counts)

                    # Case 5
                    if not mask[index+2] == '_':
                        if not mask[index+1] == '_':
                            if not mask[index-1] == '_':
                                if not mask[index-2] == '_':
                                    if not mask[index-3] == '_':
                                        if not mask[index-4] == '_':
                                            prob5 = ngram_prob(mask[index-4]+mask[index-3]+mask[index-2]+mask[index-1]+mask[index+1]+mask[index+2], char, sevengram_counts_fifth)
                                        else:
                                            prob5 = ngram_prob(mask[index-3]+mask[index-2]+mask[index-1]+mask[index+1]+mask[index+2], char, sixgram_counts_fourth)
                                    else:
                                        prob5 = ngram_prob(mask[index-2]+mask[index-1]+mask[index+1]+mask[index+2], char, fivegram_counts_third)
                                else:
                                    prob5 = ngram_prob(mask[index-1]+mask[index+1]+mask[index+2], char, fourgram_counts_second)
                            else:
                                prob5 = ngram_prob(mask[index+1]+mask[index+2], char, trigram_counts_first)
                        else:
                            if not mask[index-1] == '_':
                                if not mask[index-2] == '_':
                                    if not mask[index-3] == '_':
                                        if not mask[index-4] == '_':
                                            prob5 = ngram_prob(mask[index-4]+mask[index-3]+mask[index-2]+mask[index-1], char, fivegram_counts_fifth)
                                        else:
                                            prob5 = ngram_prob(mask[index-3]+mask[index-2]+mask[index-1], char, fourgram_counts_fourth)
                                    else:
                                        prob5 = ngram_prob(mask[index-2]+mask[index-1], char, trigram_counts_third)
                                else:
                                    prob5 = ngram_prob(mask[index-1], char, bigram_counts_second)
                            else:
                                prob5 = ngram_prob(n, char, unigram_counts)
                    else:
                        if not mask[index+1] == '_':
                            if not mask[index-1] == '_':
                                if not mask[index-2] == '_':
                                    if not mask[index-3] == '_':
                                        if not mask[index-4] == '_':
                                            prob5 = ngram_prob(mask[index-4]+mask[index-3]+mask[index-2]+mask[index-1]+mask[index+1], char, sixgram_counts_fifth)
                                        else:
                                            prob5 = ngram_prob(mask[index-3]+mask[index-2]+mask[index-1]+mask[index+1], char, fivegram_counts_fourth)
                                    else:
                                        prob5 = ngram_prob(mask[index-2]+mask[index-1]+mask[index+1], char, fourgram_counts_third)
                                else:
                                    prob5 = ngram_prob(mask[index-1]+mask[index+1], char, trigram_counts_second)
                            else:
                                prob5 = ngram_prob(mask[index+1], char, bigram_counts_first)
                        else:
                            if not mask[index-1] == '_':
                                if not mask[index-2] == '_':
                                    if not mask[index-3] == '_':
                                        if not mask[index-4] == '_':
                                            prob5 = ngram_prob(mask[index-4]+mask[index-3]+mask[index-2]+mask[index-1], char, fivegram_counts_fifth)
                                        else:
                                            prob5 = ngram_prob(mask[index-3]+mask[index-2]+mask[index-1], char, fourgram_counts_fourth)
                                    else:
                                        prob5 = ngram_prob(mask[index-2]+mask[index-1], char, trigram_counts_third)
                                else:
                                    prob5 = ngram_prob(mask[index-1], char, bigram_counts_second)
                            else:
                                prob5 = ngram_prob(n, char, unigram_counts)

                    # Case 6
                    if not mask[index+1]  == '_':
                        if not mask[index-1] == '_':
                            if not mask[index-2] == '_':
                                if not mask[index-3] == '_':
                                    if not mask[index-4] == '_':
                                        if not mask[index-5] == '_':
                                            prob6 = ngram_prob(mask[index-5]+mask[index-4]+mask[index-3]+mask[index-2]+mask[index-1]+mask[index+1], char, sevengram_counts_sixth)
                                        else:
                                            prob6 = ngram_prob(mask[index-4]+mask[index-3]+mask[index-2]+mask[index-1]+mask[index+1], char, sixgram_counts_fifth)
                                    else:
                                        prob6 = ngram_prob(mask[index-3]+mask[index-2]+mask[index-1]+mask[index+1], char, fivegram_counts_fourth)
                                else:
                                    prob6 = ngram_prob(mask[index-2]+mask[index-1]+mask[index+1], char, fourgram_counts_third)
                            else:
                                prob6 = ngram_prob(mask[index-1]+mask[index+1], char, trigram_counts_second)
                        else:
                            prob6 = ngram_prob(mask[index+1], char, bigram_counts_first)
                    else:
                        if not mask[index-1] == '_':
                            if not mask[index-2] == '_':
                                if not mask[index-3] == '_':
                                    if not mask[index-4] == '_':
                                        if not mask[index-5] == '_':
                                            prob6 = ngram_prob(mask[index-5]+mask[index-4]+mask[index-3]+mask[index-2]+mask[index-1], char, sixgram_counts_sixth)
                                        else:
                                            prob6 = ngram_prob(mask[index-4]+mask[index-3]+mask[index-2]+mask[index-1], char, fivegram_counts_fifth)
                                    else:
                                        prob6 = ngram_prob(mask[index-3]+mask[index-2]+mask[index-1], char, fourgram_counts_fourth)
                                else:
                                    prob6 = ngram_prob(mask[index-2]+mask[index-1], char, trigram_counts_third)
                            else:
                                prob6 = ngram_prob(mask[index-1], char, bigram_counts_second)
                        else:
                            prob6 = ngram_prob(n, char, unigram_counts)

                    # Case 7
                    if not mask[index-1] == '_':
                        if not mask[index-2] == '_':
                            if not mask[index-3] == '_':
                                if not mask[index-4] == '_':
                                    if not mask[index-5] == '_':
                                        if not mask[index-6] == '_':
                                            prob7 = ngram_prob(mask[index-6]+mask[index-5]+mask[index-4]+mask[index-3]+mask[index-2]+mask[index-1], char, sevengram_counts_seventh)
                                        else:
                                            prob7 = ngram_prob(mask[index-5]+mask[index-4]+mask[index-3]+mask[index-2]+mask[index-1], char, sixgram_counts_sixth)
                                    else:
                                        prob7 = ngram_prob(mask[index-4]+mask[index-3]+mask[index-2]+mask[index-1], char, fivegram_counts_fifth)
                                else:
                                    prob7 = ngram_prob(mask[index-3]+mask[index-2]+mask[index-1], char, fourgram_counts_fourth)
                            else:
                                prob7 = ngram_prob(mask[index-2]+mask[index-1], char, trigram_counts_third)
                        else:
                            prob7 = ngram_prob(mask[index-1], char, bigram_counts_second)
                    else:
                        prob7 = ngram_prob(n, char, unigram_counts)

                    # Choose max prob of fivegram first, second, third, fourth and fifth
                    char_prob += max(prob1, prob2, prob3, prob4, prob5, prob6, prob7)

                # The final case is that the character is guessed so we skip this position
                else:
                    continue

            sevengram_probs.append(char_prob)

        guess_letter = '!'

        if max(sevengram_probs) > 0 :
            guess_letter = available[sevengram_probs.index(max(sevengram_probs))]
        else:
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break  

        return guess_letter
        

        




    def my_guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary


                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [7]:
api = HangmanAPI(access_token="974bf1caaa626e9807e7077c3a4dec", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [ ]:
for i in range(10):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 8c1331dd3a3f. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '8c1331dd3a3f', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': '8c1331dd3a3f', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ i _ _ '}
Guessing letter: s
Sever response: {'game_id': '8c1331dd3a3f', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ _ _ i _ _ '}
Guessing letter: n
Sever response: {'game_id': '8c1331dd3a3f', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ _ _ _ i _ _ '}
Guessing letter: a
Sever response: {'game_id': '8c1331dd3a3f', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ a _ _ i a _ '}
Guessing letter: l
Sever response: {'game_id': '8c1331dd3a3f', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ a _ _ i a l '}
Guessing letter: r
Sever response: {'game_id': '8c1331dd3a3f', 'status': 'ong

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [20]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game


HangmanAPIError: {'error': 'Your account has been deactivated!'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [21]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print(total_recorded_runs)
print('overall success rate = %.3f' % success_rate)

1000
overall success rate = 0.625
